# SLIM runbook

In this notebook we run and save SLIM recommendations according to the best known model configuration.

## Loading libraries and setting parameters

In [ ]:
import implicit
import sys
from time import time

sys.path.append("../../..")

from src.common import config
from src.models.slim import SLIM
from src.evaluation.evaluator import Evaluator

%load_ext autoreload

In [2]:
MODEL_NAME = "slim"

paths = config.Paths(
    dataset_name="jobs_published-part",
    target_users_name="subset_validation",
    model_name=MODEL_NAME,
)

In [3]:
N_RECOMMENDATIONS = 10

## Model training

In [4]:
%autoreload
model = SLIM()

model.load_interactions(paths.train)

model.preprocess()

start = time()
model.fit()
print(time() - start)

12392it [00:31, 391.92it/s]


31.78698706626892


## Generating recommendations

In [5]:
target_users = model.load_target_users(paths.target_users)

recommendations = model.recommend(
    target_users=target_users, n_recommendations=N_RECOMMENDATIONS
)

model.save_recommendations(recommendations, paths.recommendations)

17857it [00:05, 3554.07it/s]


## Evaluation

In [6]:
%autoreload
evaluator = Evaluator(
    recommendations_path=paths.recommendations_folder,
    test_path=paths.validation,
    k=10,
    models_to_evaluate=[MODEL_NAME],
)

evaluator.prepare()

evaluator.evaluate_models()

17857it [00:01, 16537.69it/s]


In [7]:
evaluator.evaluation_results

,precision,recall,F_1,F_05,ndcg,mAP,MRR,LAUC,HR,reco_in_test,test_coverage,Shannon,Gini,users_without_reco,users_without_k_reco
model_name,,,,,,,,,,,,,,,
slim,0.017019,0.087528,0.026505,0.019737,0.062513,0.044589,0.074025,0.543119,0.146889,0.612544,0.344285,7.314275,0.842918,0.152713,0.477124
